## Slack API 연결

In [1]:
import os, requests
from pathlib import Path
from dotenv import load_dotenv
from slack_bolt import App
from slack_bolt.adapter.socket_mode import SocketModeHandler

In [2]:
load_dotenv('../.env')
SLACK_BOT_TOKEN = os.getenv("SLACK-BOT")
CHANNEL_ID = 'C086KU2133M' # Channel ID
SLACK_APP_TOKEN = os.getenv("SLACK-APP")

In [3]:
# Slack 앱을 초기화합니다
app = App(token=SLACK_BOT_TOKEN)

In [4]:
current_path = Path.cwd()

In [5]:
def download_file(file_url, file_name):   # 파일을 다운로드합니다
    response = requests.get(file_url, headers={"Authorization": f"Bearer {SLACK_BOT_TOKEN}"})
    if response.status_code == 200:
        # 파일을 저장합니다
        with open(file_name, "wb") as f:
            f.write(response.content)
        return True
    else:
        return False

In [6]:
# 모든 이벤트를 로깅하는 미들웨어
@app.middleware
def log_request(logger, body, next):
    print(f"이벤트 수신: {body['event']['type']}")
    next()

In [7]:
# 파일이 공유될 때 실행되는 이벤트 리스너
@app.event("file_shared")
def handle_file_shared(event, say, logger, client):
    # 이미지 파일인지 확인합니다
    file_id = event["file_id"]
    logger.info(f"새로운 이미지가 공유되었습니다. 파일 ID: {file_id}")
    # 파일 정보를 가져옵니다
    file_info = client.files_info(file=file_id)
    file = file_info["file"]
    file_url = file["url_private_download"]
    file_name = current_path / '../image' / file["name"]
    if download_file(file_url, file_name):
        say(f"파일 '{file_name}'을 성공적으로 다운로드했습니다.")
        # langchain으로 정보 처리
        
        # receipt_info = extract_receipt_info(file_name)
        # 추출된 정보를 바탕으로 카테고리 판단
        
    else:
        say(f"파일 다운로드에 실패했습니다.")

In [8]:
# message에 대한 처리는 필요 없음
@app.event("message")
def handle_message_events(event, logger, say):
    # 메시지 텍스트 추출
    message_text = event["text"]
    # 메시지 발신자 ID 추출
    user_id = event["user"]
    # 채널 ID 추출
    channel_id = event["channel"]
    # 추출한 정보 사용 예시
    say(f"<@{user_id}>님이 '{message_text}' 메시지를 보냈습니다.")
    logger.info(f"메시지 수신: {message_text}")

In [9]:
# 에러 핸들러
@app.error
def custom_error_handler(error, body, logger):
    print(f"에러 발생: {error}")

In [ ]:
handler = SocketModeHandler(app, SLACK_APP_TOKEN)
handler.start()